## Libraries

In [16]:
#from accelerate import PartialState
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
import torch

## Creating 4-bit Quantization Config

In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

## DeepSeek-Coder-V2-Lite-Instruct

In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct")

model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct:
- configuration_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct:
- modeling_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [03:06<00:00, 46.57s/it]


## CodeLlama-13b-Instruct-hf (and 7B)

In [18]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    quantization_config=bnb_config, #Model quantization
    trust_remote_code=True,
    low_cpu_mem_usage=True, #Automatic with quantized models
)

# del(model)
# del(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
model.to("cuda:2")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [7]:
prompt = 'from typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool: \n""" Check if in given list of numbers, are any two numbers closer to each other than given threshold. >>> has_close_elements([1.0, 2.0, 3.0], 0.5) False >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) True """'

In [10]:
prompt = r'''Generate Unit Tests.

import java.util.*;
import java.lang.*;
import static org.junit.jupiter.api.Assertions.*;
import org.junit.jupiter.api.Test;


class Solution {
    public boolean hasCloseElements(List<Double> numbers, double threshold) {
        for (int i = 0; i < numbers.size(); i++) {
            for (int j = i + 1; j < numbers.size(); j++) {
                double distance = Math.abs(numbers.get(i) - numbers.get(j));
                if (distance < threshold) return true;
            }
        }
        return false;
    }
}
        
public class SolutionTest {'''

In [12]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [11]:
inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda:2")
outputs = model.generate(inputs, 
                         num_beams = 2, 
                         num_beam_groups = 2,
                         do_sample = False, 
                         max_new_tokens = 500,
                         top_p = 1,
                         top_k = 50,
                         #penalty_alpha = 0.5,
                         length_penalty = 1.0,
                         diversity_penalty = 0.5
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generate Unit Tests.

import java.util.*;
import java.lang.*;
import static org.junit.jupiter.api.Assertions.*;
import org.junit.jupiter.api.Test;


class Solution {
    public boolean hasCloseElements(List<Double> numbers, double threshold) {
        for (int i = 0; i < numbers.size(); i++) {
            for (int j = i + 1; j < numbers.size(); j++) {
                double distance = Math.abs(numbers.get(i) - numbers.get(j));
                if (distance < threshold) return true;
            }
        }
        return false;
    }
}
        
public class SolutionTest {
    @Test
    public void test1() {
        List<Double> numbers = new ArrayList<Double>();
        numbers.add(1.0);
        numbers.add(2.0);
        numbers.add(3.0);
        numbers.add(4.0);
        numbers.add(5.0);
        assertTrue(new Solution().hasCloseElements(numbers, 1.5));
    }

    @Test
    public void test2() {
        List<Double> numbers = new ArrayList<Double>();
        numbers.add(1.0);
        n

## SantaCoder

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigcode/santacoder", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "bigcode/santacoder",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)
del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/bigcode/santacoder:
- configuration_gpt2_mq.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/bigcode/santacoder:
- modeling_gpt2_mq.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## Instruct CodeT5P - 16b params

In [7]:
from transformers import AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/instructcodet5p-16b")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Salesforce/instructcodet5p-16b",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/Salesforce/instructcodet5p-16b:
- modeling_codet5p.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
CodeT5pForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an e

## CodeT5p-770m (Alternative to CodeT5p)(https://huggingface.co/Salesforce/codet5p-770m)

In [9]:
from transformers import AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-6b")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Salesforce/codet5p-6b",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## CodeT5-Base - 770M params (Alternative to CodeT5p)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Salesforce/codet5-base",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## CodeGen2.5-7b-Instruct_P

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen25-7b-instruct_P")
model = AutoModelForCausalLM.from_pretrained(
    "Salesforce/codegen25-7b-instruct_P",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## CodeGen2 - 1B params (Alternative to codegen 2.5)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen2-1B_P", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "Salesforce/codegen2-1B_P",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/Salesforce/codegen2-1B_P:
- configuration_codegen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codegen2-1B_P:
- modeling_codegen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## WizardCoder-15B-V1.0

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("WizardLMTeam/WizardCoder-15B-V1.0")
model = AutoModelForCausalLM.from_pretrained(
    "WizardLMTeam/WizardCoder-15B-V1.0",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## StarCoder2-3b

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained(
    "bigcode/starcoder2-3b",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## Freeing Up Cache Space

In [ ]:
#Delete models from cache
import subprocess

subprocess.run(["rm", "-rf", "~/.cache/huggingface/hub"])

In [ ]:
#Delete datasets from cache (Shouldn't be needed with this data)
import subprocess

subprocess.run(["rm", "-rf", "~/.cache/huggingface/datasets"])